In [1]:
import sys
sys.path.append('C:\\users\\yoyoy\\anaconda3\\envs\\rl\\lib\\site-packages')

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import cv2 
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Read all images and convert them to array

In [12]:
Datadirectory = "dataset/train/"
Classes = ["angry","disgust","fear","happy","neutral","sad","surprise"]
img_size = 224 # ImageNet -> 224 x 224

training_data = []

def create_training_data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                image = cv2.imread(os.path.join(path, img))
                #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (img_size, img_size))
                training_data.append([image, class_num])
            except Exception as e:
                pass

In [13]:
create_training_data()

In [ ]:
#print(len(training_data))

In [ ]:
#temp = np.array(training_data)
#temp.shape
#temp[0][1]

In [14]:
random.shuffle(training_data) #shuffle data

In [15]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, img_size, img_size, 3)
y = np.array(y)

In [16]:
X.shape

(12475, 224, 224, 3)

In [17]:
#Y = np.array(y)
y.shape

(12475,)

In [18]:
training_data.clear()
#Classes.clear()
#print(len(training_data))

In [19]:
# Normalize data
# you could use scikit-learn

X = X/255.0

In [20]:
X.shape

(12475, 224, 224, 3)

### Deep Learning model for training - Transfer Learning

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import h5py

In [4]:
model = tf.keras.applications.MobileNetV2() #pre-trained model

In [ ]:
#model.summary()

### Transfer Learning - Tuning, weights will start

In [5]:
base_input = model.layers[0].input

In [6]:
base_output = model.layers[-2].output

In [7]:
#base_output

In [8]:
final_output = layers.Dense(128)(base_output) #add new layer after global pooling layer
final_output = layers.Activation('relu')(final_output) #activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output) #output is 7 classes

In [ ]:
final_output

In [9]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [10]:
#new_model.summary()

In [11]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
new_model.fit(X,y, epochs = 30)

Epoch 1/30
390/390 [==============================] - 1033s 3s/step - loss: 1.4239 - accuracy: 0.4547
Epoch 2/30
390/390 [==============================] - 974s 2s/step - loss: 1.2228 - accuracy: 0.5343
Epoch 3/30
390/390 [==============================] - 947s 2s/step - loss: 1.1275 - accuracy: 0.5772
Epoch 4/30
390/390 [==============================] - 945s 2s/step - loss: 1.0509 - accuracy: 0.6064
Epoch 5/30
390/390 [==============================] - 889s 2s/step - loss: 0.9898 - accuracy: 0.6305
Epoch 6/30
390/390 [==============================] - 877s 2s/step - loss: 0.9192 - accuracy: 0.6556
Epoch 7/30
390/390 [==============================] - 884s 2s/step - loss: 0.8659 - accuracy: 0.6794
Epoch 8/30
390/390 [==============================] - 887s 2s/step - loss: 0.7984 - accuracy: 0.7024
Epoch 9/30
390/390 [==============================] - 880s 2s/step - loss: 0.7320 - accuracy: 0.7301
Epoch 10/30
390/390 [==============================] - 862s 2s/step - loss: 0.6792 - accur

In [22]:
new_model.save('FER13_Accuracy95.h5')